# Load Data

In [ ]:
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install soundfile
!pip install jiwer

  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-e307m1bz
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-e307m1bz
     |████████████████████████████████| 245kB 8.6MB/s 
     |████████████████████████████████| 112kB 29.1MB/s 
  Created wheel for datasets: filename=datasets-1.5.0.dev0-cp37-none-any.whl size=192875 sha256=bbc2155c5b87d1009969ad0f9b84f69c512e5a7206c86cab2a75882791dd1cf0
  Stored in directory: /tmp/pip-ephem-wheel-cache-gf1k1ojb/wheels/3e/af/ff/d1cdb5d0f9cff6eba2042a16b477ada497e23f1a3b6950b928
Successfully built datasets
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-nq89y0o3
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-nq89y0o3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.2MB 8.7MB/s 
     |███

In [ ]:
from datasets import load_dataset, load_metric

data = load_dataset("csv",data_files={'train': ['data .csv'],
                                              'test': ['test_data .csv']})


Using custom data configuration default-5ddb664b6908ecc0


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5ddb664b6908ecc0/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'file', 'text'],
        num_rows: 60
    })
    test: Dataset({
        features: ['Unnamed: 0', 'file', 'text'],
        num_rows: 3
    })
})

In [ ]:
data = data.remove_columns(["Unnamed: 0"])

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['file', 'text'],
        num_rows: 60
    })
    test: Dataset({
        features: ['file', 'text'],
        num_rows: 3
    })
})

## Removing special caracters

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
data = data.map(remove_special_characters)

## extract vocab

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocabs = data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=data.column_names["train"])

In [ ]:
vocabs["train"]["all_text"]

["hello my name is yasmine i was born in nineteen ninety eight   i'm a student at national school of engineering of carthage   i'm from tunisia   my father my mother my sister my brother alvia   what's the weather like today let's go camping   i'm happy i'm excited   welcome evreyone today we talked about speechrecognition   good morning evreyone and welcome to my presentation first of all let me thank you all for coming here today in my presentation i'm gonna talk about the powerful subfield of ai called speech recognition i'll walk you through what is this field and how it works   i didn't have dinner   do you mind giving me your number   yes please   let's travel to paris london new york   my mates are very cool kind   let's go shopping and buy some stuff   how are you i guess you're fine   i really had a hard time dealing with this project   thank you for your help i really appreciate that   how about you   are you okey   good morning good afternoon   i'll see you soon   thank you 

In [ ]:
vocabs["train"]["vocab"]

[['d',
  'p',
  'i',
  'g',
  'v',
  '’',
  'w',
  'c',
  't',
  'o',
  'y',
  'a',
  'j',
  'h',
  'q',
  'u',
  'z',
  'e',
  'n',
  ' ',
  'l',
  'f',
  'k',
  "'",
  'r',
  's',
  'b',
  'x',
  'm']]

In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [ ]:
len(vocab_list)

29

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 19,
 "'": 23,
 'a': 11,
 'b': 26,
 'c': 7,
 'd': 0,
 'e': 17,
 'f': 21,
 'g': 3,
 'h': 13,
 'i': 2,
 'j': 12,
 'k': 22,
 'l': 20,
 'm': 28,
 'n': 18,
 'o': 9,
 'p': 1,
 'q': 14,
 'r': 24,
 's': 25,
 't': 8,
 'u': 15,
 'v': 4,
 'w': 6,
 'x': 27,
 'y': 10,
 'z': 16,
 '’': 5}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
del vocab_dict["’"]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

30

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

# Wav2Vec2CTCTokenizer

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

# Wav2Vec2FeatureExtractor



In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

# Wav2Vec2Processor

In [ ]:

from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
 from google.colab import drive
 drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
 processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-mybase-timit-demo")

In [ ]:
data["train"][0]

{'file': '/content/gdrive/MyDrive/asr _dataset/1.wav',
 'text': 'hello my name is yasmine i was born in nineteen ninety eight  '}

# Data preprocessing

In [ ]:
import soundfile as sf
import librosa
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = librosa.load(batch["file"],sr=16000)
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

In [ ]:
data =data.map(speech_file_to_array_fn, remove_columns=data.column_names["train"], num_proc=3)

In [ ]:
import random
import numpy as np
rand_int = random.randint(0, len(data["train"]))

print("Target text:", data["train"][rand_int]["target_text"])
print("Input array shape:", np.asarray(data["train"][rand_int]["speech"]).shape)
print("Sampling rate:", data["train"][rand_int]["sampling_rate"])

Target text: i'll see you soon  
Input array shape: (38323,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
data_prepared = data.map(prepare_dataset, remove_columns=data.column_names["train"], batch_size=8, num_proc=3, batched=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
data_prepared

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 60
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 3
    })
})

#Training and evaluation 

## Set up training

In [ ]:
#A DataCollator is a function that takes a list of samples from a Dataset and collate them into a batch, as a dictionary. of Tensors.
#defines the padding , it will dynamically pad the inputs received 
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
   

    processor: Wav2Vec2Processor
    #pad to the longest sequence in the batch 
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
   output_dir="/content/gdrive/MyDrive/wav2vec2-mybase-timit-demo",
  
  group_by_length=True,
  per_device_train_batch_size=5,
  evaluation_strategy="steps",
  num_train_epochs=30,
  
  save_steps=50,
  eval_steps=50,
  logging_steps=50,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

## training

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data_prepared["train"],
    eval_dataset=data_prepared["test"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()